## LifeStyle Disease Surveillance Using Spatio-Temporal Search Intensity Models
(c) 2017 by Shahan Ali Memon

## Goal

The goal of this research is to predict the prevalence of lifestyle diseases (such as obesity, diabetes, etc.) using Google Trends.

### Prerequisite installations
-----------------
To run this program you'd need the following python libraries
1. numpy (pip install numpy)

2. sklearn (pip install sklearn)

3. matplotlib (optional - pip install matplotlib)

4. cPickle (pip install cPickle)

### Preprocessing
*** Collecting Data and converting to CSV format ***

Our first task will be to collect data from Google Trends. You could use pytrends for that. Once the spatial and temporal data is collected and you have applied the spatio-temporal formula mentioned in our paper, you can then use the following code to preprocess the csv file to create numpy arrays.

- Hence collect the data from google trends and create a CSV file in the same format as in: Data/Raw/
- Once you have the CSV file in the format mentioned, run the preprocess.py file in Data/Code/ as follows:
- python preprocess.py <filepath_to_state_list> <filepath_to_csv_file> <directory_path_to_save_numpy_arrays> <comma_separated_training_years> <comma_separated_test_years> <feature_scaling_flag>

In [5]:
'''
Description: This piece of code is used to preprocess the
raw data from csv files to create numpy arrays for train
and test.
-------------------------------------
Author: Shahan A. Memon
Advisors: Ingmar Weber, Saquib Razak

Copyright: Carnegie Mellon University,
Qatar Computing Research Institute 2017
--------------------------------------
'''

import numpy as np
from sklearn.preprocessing import StandardScaler
import sys
import os
from sklearn import preprocessing

'''
Description: This function simply reads a file line by line
and creates a python list out of it. For our purposes it reads
the states or keywords for which we have data available.
Sometimes data for some states or keywords is missing,
and so we might not have data available for all the 52 states for example.
This is a simple list creator for a file in which data is separated by line
Requires:
- filename: file which consists of the states/keywords
for which data is available
Returns: A list of those states
'''

def readLst(filename):
    #Initialize an empty list
    lst = []
    with open(filename) as f:
        for line in f:
            #Read each line, remove whitespaces and
            #append to list
            lst.append(line.rstrip())

    #return sorted list
    return sorted(lst)

'''
This function reads the training and testing data
from a csv file to a numpy array
REQUIRES:
- states (list of states)
- fpcsv (filepath to the csv file)
NOTE: the csv file format should match our format
PLEASE LOOK into Data/Raw/ to see the file format
of our csv files
- trn_years (list of training years)
- tst_years (list of testing years)
- scale (whether to scale the data or not and how)
RETURNS:
A tuple of (features,x_train,y_train,x_test,y_test)
'''

def readCSV(states,fpcsv,trn_years,tst_years,scale):
    #Read entire data as a string array 
    all_data = np.genfromtxt(fpcsv,
                             dtype=None,
                             delimiter=',')
    '''
    Now we read the labels which are supposed to be in the 3rd col
    '''
    Y = all_data[:,[2]].flatten()[1:].astype(np.float)
    X = all_data[1:,3:].astype(np.float)
    if(scale == 1):
        X = X / X.max(axis=0)
    features = all_data[[0],3:].flatten()
    '''
    Now that we have X and Y, let's divide them into years
    #This code is assuming that years in the csv file are
    in ascending order
    '''
    year_to_arrayX = {}
    year_to_arrayY = {}
    numberOfYears = len(X[:,[0]])/len(states)
    assert(numberOfYears == len(X[:,[0]])*1.0/len(states))
    all_years = [2011+i for i in range(numberOfYears)]
    assert(set(trn_years) <= set(all_years))
    assert(set(tst_years) <= set(all_years))
    for i in range(numberOfYears):
        year_to_arrayX[all_years[i]] = X[i*len(states):(i+1)*len(states)]
        year_to_arrayY[all_years[i]] = Y[i*len(states):(i+1)*len(states)]
    #Random initialization
    x_train = year_to_arrayX[trn_years[0]]
    y_train = year_to_arrayY[trn_years[0]]
    for y in trn_years[1:]:
        x_train = np.concatenate((x_train,year_to_arrayX
                                  [y]), axis=0)
        y_train = np.concatenate((y_train,year_to_arrayY
                                  [y]), axis=0)
    #Random initialization
    x_test = year_to_arrayX[tst_years[0]]
    y_test = year_to_arrayY[tst_years[0]]
    for y in tst_years[1:]:
        x_test = np.concatenate((x_test,year_to_arrayX
                                  [y]), axis=0)
        y_test = np.concatenate((y_test,year_to_arrayY
                                  [y]), axis=0)
    if(scale == 2):
        scaler = preprocessing.StandardScaler().fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
    if(scale == 3):
        x_train = preprocessing.scale(x_train)
        x_test = preprocessing.scale(x_test)
    
    return (x_train,y_train,x_test,y_test,features)
    
    
if __name__ == "__main__":
    argv = sys.argv[1:]
    if(len(argv)== 6):
        assert(len(argv) == 6)
        fps = str(argv[0])
        fpcsv = str(argv[1])
        fpo = str(argv[2])
        trn_years = []
        tst_years = []
        scaleFlag = 1
        #Checking if file paths and dirs exist
        if(os.path.isdir(fpo) and os.path.isfile(fps) and
           os.path.isfile(fpcsv)):
            #list of train and test years
            try:
                trn_years =  map(int,str(argv[3]).split(","))
                tst_years =  map(int,str(argv[4]).split(","))
            except:
                print("Error reading training/testing years")
            try:
                scaleFlag = int(argv[5])
                assert(scaleFlag in [0,1,2,3])
            except:
                print("invalid scaling choice")
            states = readLst(fps)
            #readCSV will return a tuple of 4 numpy arrays
            x_train,y_train,x_test,y_test,features = readCSV(states,
                                                    fpcsv,
                                                    sorted(trn_years),
                                                    sorted(tst_years),
                                                    scaleFlag)
            #Saving the numpy arrays
            try:
                np.save(fpo+"/"+"x_train", x_train)
                np.save(fpo+"/"+"y_train", y_train)
                np.save(fpo+"/"+"x_test", x_test)
                np.save(fpo+"/"+"y_test", y_test)
                np.save(fpo+"/"+"features",features)
                print("Saved Arrays successfully in the folder:"+fpo)
            except:
                print("Error saving arrays")
        else:
            print("File/Directory Path Error")
        
    else:
        assert(len(argv) != 6)
        print("This program needs 6 arguments\n"\
              "Usage: python preprocess.py <fp-s>"\
              "<fp-csv> <fp-o> <train-years> <test-years>"\
              "<scale-flag>\n"\
              "fp-s: file path to the list of states\n"\
              "fp-csv: file path to the csv file of data\n"\
              "fp-o: directory path to save numpy arrays\n"\
              "train-years: comma separated years, e.g. 2011,2012\n"\
              "test-years: comma separated years, e.g. 2013,2014\n"\
              "scale-flag:\n"\
              "- 0 for no normalization\n"\
              "- 1 for max normalization\n"\
              "- 2 for mean normalization\n"\
              "- 3 mean normalize test separately from train")


This program needs 6 arguments
Usage: python preprocess.py <fp-s><fp-csv> <fp-o> <train-years> <test-years><scale-flag>
fp-s: file path to the list of states
fp-csv: file path to the csv file of data
fp-o: directory path to save numpy arrays
train-years: comma separated years, e.g. 2011,2012
test-years: comma separated years, e.g. 2013,2014
scale-flag:
- 0 for no normalization
- 1 for max normalization
- 2 for mean normalization
- 3 mean normalize test separately from train


### Training

After Preprocessing, you would then need to train the model using the numpy arrays created via the program above.
To do that you would have to run the program below as follows:

python train.py <path_to_x_train.npy> <path_to_y_train.npy> <no_of_folds_of_CV> <loss> <fname_for_model.pkl>


In [6]:
'''
Description: This piece of code is train the model
and save those models for testing
-------------------------------------
Author: Shahan A. Memon
Advisors: Ingmar Weber, Saquib Razak

Copyright: Carnegie Mellon University,
Qatar Computing Research Institute 2017
--------------------------------------
'''

import csv
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import random
import sys
import math
import cPickle

from sklearn import preprocessing
import matplotlib.pyplot as plt

'''
Description: Given numpy array x_train and y_train, this
function will train a linear regression model
REQUIRES:
- regularization: flag which will tell whether we should regularize or not
- loss function: This can be MAE,MSE,R
- cross_validation: flag which will tell if we should do cross-validation
- folds: number of folds for cross-validation
- x_train: a loaded numpy design matrix
- y_train: a loaded numpy label array
RETURNS:
This function basically returns a lasso regression model to be saved for later
and it also returns scores to be used for cross-validation
'''

def train_LASSOmodel(x_train,y_train,loss,regularization=True,
                cross_validation=True,folds=10,
                ):
    if(loss == 0):
        loss = 'neg_mean_absolute_error'
    elif(loss == 1):
        loss = 'neg_mean_squared_error'
    elif(loss == 2):
        loss = 'r2'
    model = linear_model.Lasso(random_state=0)
    #First we need to find a regularization hyper-parameter
    alphas = np.arange(0.001,1,0.001)
    scores = list()
    scores_std = list()
    counter = 0
    for alpha in alphas:
        model.alpha = alpha
        this_scores = cross_val_score(model, x_train, y_train,
                                      cv = folds, n_jobs = 1,
                                      scoring = loss)
        scores.append(np.mean(this_scores))
        scores_std.append(np.std(this_scores))
        counter += 1
    opt = max(scores)
    print(scores)
    
    indexLst = [i for i,j in enumerate(scores) if j == opt]
    alphaIndex = indexLst[0]
    optAlpha = alphas[alphaIndex]
    print("Optimal Alpha:"+str(optAlpha))
    print ("score == " + str(max(scores)))
    #So now that we have figured out the alpha
    model = linear_model.Lasso(alpha=optAlpha)
    model.fit(x_train,y_train)
    return model    


if __name__ == "__main__":
    argv = sys.argv[1:]
    if(len(argv) == 5):
        x_train = np.load(argv[0])
        y_train = np.load(argv[1])
        folds = int(argv[2])
        loss = int(argv[3])
        fname = str(argv[4])
        model = train_LASSOmodel(x_train,y_train,loss,regularization=True,
                cross_validation=True,folds=10)
        with open(fname,'wb') as fid:
            cPickle.dump(model,fid)
        '/Users/samemon/desktop/JMIR_Lifestyle_Disease_Surveillance/Train/Models/train_model.pkl'
    else:
        print("Usage: \npython train.py <path to x_train.npy>"\
              "<path to y_train.npy> <no of folds of CV> <loss> <fname>\n"\
              "where loss = 0 if MAE, 1 if MSE and 2 if r2\n"\
              "fname is path+name+.pkl for model to be saved")

Usage: 
python train.py <path to x_train.npy><path to y_train.npy> <no of folds of CV> <loss> <fname>
where loss = 0 if MAE, 1 if MSE and 2 if r2
fname is path+name+.pkl for model to be saved


### Testing

Once the models have been learnt, you can test the models in terms of their MSE, SMAPE and R correlation as follows:
python test.py <fp-test_x> <fp-test_y> <fp-model>

In [8]:
'''
Description: This piece of code is to test the model
on a provided test set
-------------------------------------
Author: Shahan A. Memon
Advisors: Ingmar Weber, Saquib Razak

Copyright: Carnegie Mellon University,
Qatar Computing Research Institute 2017
--------------------------------------
'''

import csv
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import random
import sys
import math
import cPickle

from sklearn import preprocessing
import matplotlib.pyplot as plt

def findMSE(true,pred):
    return np.mean((true-pred)**2)

def findCorr(true,pred):
    return np.corrcoef(true,pred)[[0],[1]][0]

def findSMAPE(true,pred):
    SMAPE = 0
    ratio = []
    for index in range(len(true)):
        sub = abs(true[index]-pred[index])
        avg = (true[index]+pred[index])/2.0
        ratio.append(sub*1.0/avg)
    SMAPE = sum(ratio)/len(true)*100.0
    return SMAPE

if __name__ == "__main__":
    argv = sys.argv[1:]
    if(len(argv) == 3):
        x_test = np.load(argv[0])
        y_test = np.load(argv[1])
        model = linear_model.Lasso(random_state=0)
        with open(str(argv[2]),'rb') as fid:
            model = cPickle.load(fid)
        preds = np.array(model.predict(x_test))
        MSE = findMSE(y_test,preds)
        R = findCorr(y_test,preds)
        SMAPE = findSMAPE(y_test,preds)
        print("MSE ====== :" + str(MSE))
        print("R ====== :" + str(R))
        print("SMAPE ====== :" + str(SMAPE))
        halfPreds = np.split(preds,2)
        halfTrues = np.split(y_test,2)
        MSE = (findMSE(halfTrues[0],halfPreds[0]) +
               findMSE(halfTrues[1],halfPreds[1]))/2.0
        R = (findCorr(halfTrues[0],halfPreds[0]) +
               findCorr(halfTrues[1],halfPreds[1]))/2.0
        SMAPE = (findSMAPE(halfTrues[0],halfPreds[0]) +
               findSMAPE(halfTrues[1],halfPreds[1]))/2.0
        print("1/2MSE ====== :" + str(MSE))
        print("1/2R ====== :" + str(R))
        print("1/2SMAPE ====== :" + str(SMAPE))
        
    else:
        print("Usage: <fp-test_x> <fp-test_y> <fp-model>")
        

Usage: <fp-test_x> <fp-test_y> <fp-model>
